In [ ]:
# Import packages
import pandas as pd
import numpy as np
import json
import requests

# Initialize variables
alpha_vantage_key = "" 
stock = ["AAPL"] # Just 1 for example purposes
df = pd.DataFrame()

# For loop for API call
for s in stock:
    request = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={s}&interval=1min&outputsize=full&apikey={alpha_vantage_key}").json()
    request = pd.DataFrame(request["Time Series (1min)"]).transpose().reset_index().rename(columns = {"index": "timestamp"}).assign(stock = s)
    request["timestamp"] = pd.to_datetime(request["timestamp"])
    df = df.append(request)

# Separate timestamp to date & time columns
dates, times = zip(*[(d.date(), d.time()) for d in df["timestamp"]])
df = df.assign(date = dates, time = times)

# Convert type to float
df[["1. open", "4. close"]] = df[["1. open", "4. close"]].astype(float)

# Function to calculate difference
def calculate_diff(row):
    if row["1. open"] - row["4. close"] > 0:
        return "Open > Close"
    elif row["1. open"] - row["4. close"] < 0:
        return "Open < Close"
    else:
        return "Open = Close"
df["open/close difference"] = df.apply(calculate_diff, axis = 1)

# Save to csv file
df.to_csv(r"File\\Path\\File.csv")